In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [2]:
os.listdir('archive (1)/retail_product_checkout/Data')

['alcohol',
 'candy',
 'canned_food',
 'chocolate',
 'dessert',
 'dried_food',
 'dried_fruit',
 'drink',
 'gum',
 'instant_drink',
 'instant_noodles',
 'milk',
 'personal_hygiene',
 'puffed_food',
 'seasoner',
 'stationery',
 'tissue']

### Now we will create a function to convert images into arrays

In [29]:
X = []
y = []

In [30]:
def convert(path):
        cat = 0
        for p in os.listdir(path):
            new = os.path.join(path,p)
            for i in os.listdir(new):
                img_array = cv2.imread(os.path.join(new,i),cv2.IMREAD_GRAYSCALE)
                new_img_array = cv2.resize(img_array, dsize=(80, 80))
                X.append(new_img_array)
                y.append(cat) 
            cat = cat + 1
            
       

In [32]:
convert('archive (1)/retail_product_checkout/Data')

In [33]:
t = set(y)
t

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}

In [34]:
len(y)

53738

In [35]:
X = np.array(X).reshape(-1, 80,80,1)
y = np.array(y)

Normalize the data

In [36]:
X = X/255

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=42)

In [38]:
len(X_train)

42990

In [41]:
model = Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(Conv2D(64,(3,3), activation = 'relu', input_shape = X_train.shape[1:]))
model.add(MaxPooling2D(pool_size = (2,2)))
# Add another:
model.add(Conv2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(Dense(17, activation='softmax'))

In [42]:
model.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1075/1075 [==============================] - 327s 303ms/step - loss: 1.4123 - accuracy: 0.5242 - val_loss: 0.7880 - val_accuracy: 0.7235
Epoch 2/10
1075/1075 [==============================] - 327s 304ms/step - loss: 0.5599 - accuracy: 0.8001 - val_loss: 0.4637 - val_accuracy: 0.8302
Epoch 3/10
1075/1075 [==============================] - 365s 340ms/step - loss: 0.3308 - accuracy: 0.8836 - val_loss: 0.2911 - val_accuracy: 0.8926
Epoch 4/10
1075/1075 [==============================] - 346s 322ms/step - loss: 0.2181 - accuracy: 0.9217 - val_loss: 0.2176 - val_accuracy: 0.9229
Epoch 5/10
1075/1075 [==============================] - 340s 317ms/step - loss: 0.1566 - accuracy: 0.9460 - val_loss: 0.2119 - val_accuracy: 0.9261
Epoch 6/10
1075/1075 [==============================] - 370s 344ms/step - loss: 0.1225 - accuracy: 0.9574 - val_loss: 0.1758 - val_accuracy: 0.9367
Epoch 7/10
1075/1075 [==============================] - 332s 309ms/step - loss: 0.0958 - accuracy: 0.9678 - val_

In [44]:
prediction = model.predict(X_test)


In [68]:
results = np.argmax(prediction,axis=1)

In [71]:
print("model accuracy score: ", accuracy_score(y_test, results))

model accuracy score:  0.9095645701525865


In [79]:
print('F1 score: ', f1_score(y_test,results, average='macro'))

F1 score:  0.9021954238704862


In [80]:
model.save('model.h5')